In [1]:
import pandas as pd
import numpy as np
from glob import glob
import os
from pathlib import Path
from datetime import date
import json
import time
from config import *
from send_mail import send_report
from itertools import product


In [6]:
Customer_summary_Output_By_FailMonth = pd.read_excel("../Final_Output_folder/Customer_summary_Output_By_FailMonth.xlsx")
Customer_summary_Output_By_FailMonth = Customer_summary_Output_By_FailMonth[Customer_summary_Output_By_FailMonth.Fail_Month.notna()]
Customer_summary_Output_By_FailMonth.loc[:,"Totol_Cost_Sum"] = Customer_summary_Output_By_FailMonth.loc[:,"Totol_Cost_Sum"].fillna(0)
Customer_summary_Output_By_FailMonth = Customer_summary_Output_By_FailMonth[Customer_summary_Output_By_FailMonth.loc[:,"Material"].apply(lambda x:len(str(x))==12)]  # EXCLUDE ["非RBCW产品",NAN]
Customer_summary_Output_By_FailMonth.loc[:,"year"] = Customer_summary_Output_By_FailMonth.loc[:,"Fail_Month"].apply(lambda x:int(str(x)[:4]))
Customer_summary_Output_By_FailMonth.to_excel("Customer_summary_Output_By_FailMonth_detail.xlsx")
Customer_summary_Output_By_FailYear = Customer_summary_Output_By_FailMonth.dropna(subset="TNS_Year")
Customer_summary_Output_By_FailYear = Customer_summary_Output_By_FailYear.drop_duplicates(subset=["Customer_Name", "Material","year"])
Customer_summary_Output_By_FailYear = Customer_summary_Output_By_FailYear.loc[:,["Customer_Name", "year", "TNS_Year"]]
Customer_summary_Output_By_FailYear = Customer_summary_Output_By_FailYear.groupby(by=["Customer_Name","year"]).sum()
Customer_summary_Output_By_FailMonth_year_cost = Customer_summary_Output_By_FailMonth[["Customer_Name","Totol_Cost_Sum","year"]].groupby(by=["Customer_Name","year"]).sum()
Customer_summary_Output_By_FailMonth_year_edc = Customer_summary_Output_By_FailMonth_year_cost.merge(Customer_summary_Output_By_FailYear, left_index=True,right_index=True,how="right")
# Customer_summary_Output_By_FailMonth_year_edc.loc[:,"TNS"] = Customer_summary_Output_By_FailMonth_year_edc.loc[:,"TNS"].apply(lambda x:np.abs(x))
Customer_summary_Output_By_FailMonth_year_edc.loc[:,"edc"] = Customer_summary_Output_By_FailMonth_year_edc.loc[:,"Totol_Cost_Sum"]/ Customer_summary_Output_By_FailMonth_year_edc.loc[:,"TNS_Year"]
Customer_summary_Output_By_FailMonth_year_edc.loc[:,"current_year"] = date.today().year
Customer_summary_Output_By_FailMonth_year_edc.loc[:,"current_month"] = date.today().month
def filter_month(current_month : int):
    if date.today().day > 20:
        return current_month
    else:
        return current_month - 1

Customer_summary_Output_By_FailMonth_year_edc.loc[:,"current_month"] = Customer_summary_Output_By_FailMonth_year_edc.loc[:,"current_month"].apply(lambda x:filter_month(x))
Customer_summary_Output_By_FailMonth_year_edc = Customer_summary_Output_By_FailMonth_year_edc[Customer_summary_Output_By_FailMonth_year_edc.index.get_level_values("year") == Customer_summary_Output_By_FailMonth_year_edc.current_year]
Customer_summary_Output_By_FailMonth_year_edc = Customer_summary_Output_By_FailMonth_year_edc[Customer_summary_Output_By_FailMonth_year_edc.edc > EDC]
Customer_summary_Output_By_FailMonth_year_edc.loc[:,"edc"] = Customer_summary_Output_By_FailMonth_year_edc.loc[:,"edc"].apply(lambda x:float(x * 100))
Customer_summary_Output_By_FailMonth_year_edc.loc[:,"edc"] = Customer_summary_Output_By_FailMonth_year_edc.loc[:,"edc"].apply(lambda x:np.round(x,2).astype(str) + ' %')

Customer_summary_Output_By_FailMonth_year_edc.to_excel("Customer_summary_Output_By_FailMonth_year_edc.xlsx")

# for json_data in json.loads(Customer_summary_Output_By_FailMonth_year_edc.to_json(orient='records')):
#     Customer_Name = json_data["Customer_Name"]
#     current_year = json_data["current_year"]
#     current_month = json_data["current_month"]
#     edc =json_data["edc"]
#     send_report(Subject="EDC Warning",content_1=f"您好， {Customer_Name}客户,{current_year}年{current_month}月YTD EDC={edc}, 已超过公司质量目标，请调查原因及制定改善措施，谢谢。")
#     time.sleep(10)
# Customer_summary_Output_By_FailMonth_year_edc
Customer_summary_Output_By_FailMonth_year_edc

C:\Users\RBAdmin_App1\AppData\Local\Temp\4\ipykernel_7952\174294120.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0.69 %' '1.21 %' '8.33 %' '4.91 %' '1.04 %' '0.79 %' '4.31 %' '1.71 %'
 '2.55 %' '1.95 %' '3.3 %' '0.94 %' '6.82 %' '2.72 %' '35.06 %']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  Customer_summary_Output_By_FailMonth_year_edc.loc[:,"edc"] = Customer_summary_Output_By_FailMonth_year_edc.loc[:,"edc"].apply(lambda x:np.round(x,2).astype(str) + ' %')


,,Totol_Cost_Sum,TNS_Year,edc,current_year,current_month
Customer_Name,year,,,,,
BFCEC,2025,2.819498e+05,4.060044e+07,0.69 %,2025,6
CNHTC_HAEP,2025,8.770994e+04,7.221330e+06,1.21 %,2025,6
CNHTC_Jinan Power,2025,1.654416e+04,1.987200e+05,8.33 %,2025,6
DCEC,2025,1.305444e+06,2.660595e+07,4.91 %,2025,6
FAWDE,2025,1.146505e+05,1.103641e+07,1.04 %,2025,6
FAWJF,2025,1.015371e+05,1.286569e+07,0.79 %,2025,6
FOTON,2025,1.969813e+06,4.565546e+07,4.31 %,2025,6
GWM,2025,3.574619e+06,2.090962e+08,1.71 %,2025,6
JIM,2025,7.384611e+05,2.891533e+07,2.55 %,2025,6


In [ ]:
Customer_summary_Output_By_FailMonth = pd.read_excel("../Final_Output_folder/Customer_summary_Output.xlsx")
Customer_summary_Output_By_FailMonth = Customer_summary_Output_By_FailMonth.groupby([])
Customer_summary_Output_By_FailMonth = Customer_summary_Output_By_FailMonth.loc[:, Customer_summary_Output_By_FailMonth.loc[:,"Statistic month"].notna()]
Customer_summary_Output_By_FailMonth = Customer_summary_Output_By_FailMonth[Customer_summary_Output_By_FailMonth.TNS_QTY.notna()]
Customer_summary_Output_By_FailMonth = Customer_summary_Output_By_FailMonth[Customer_summary_Output_By_FailMonth.Totol_Cost_Sum.notna()]
Customer_summary_Output_By_FailMonth.loc[:,"year"] = Customer_summary_Output_By_FailMonth.loc[:,"Statistic month"].apply(lambda x:int(str(x)[:4]))
Customer_summary_Output_By_FailMonth.loc[:,"month"] = Customer_summary_Output_By_FailMonth.loc[:,"Statistic month"].apply(lambda x:int(str(x)[5:7]))
Customer_summary_Output_By_FailMonth_year_complaint = Customer_summary_Output_By_FailMonth[["Customer_Name","Product_Name","Mil<1000 Qty","year","month"]].groupby(by=["Customer_Name","Product_Name","year","month"]).sum()
Customer_summary_Output_By_FailMonth_year_complaint.loc[:,"current_year"] = date.today().year
Customer_summary_Output_By_FailMonth_year_complaint.loc[:,"current_month"] = date.today().month
def filter_month(current_month : int):
    if date.today().day > 20:
        return current_month
    else:
        return current_month - 1

Customer_summary_Output_By_FailMonth_year_complaint.loc[:,"current_month"] = Customer_summary_Output_By_FailMonth_year_complaint.loc[:,"current_month"].apply(lambda x:filter_month(x))
Customer_summary_Output_By_FailMonth_year_complaint = Customer_summary_Output_By_FailMonth_year_complaint[Customer_summary_Output_By_FailMonth_year_complaint.index.get_level_values("year") == Customer_summary_Output_By_FailMonth_year_complaint.current_year]
# Customer_summary_Output_By_FailMonth_year_complaint = Customer_summary_Output_By_FailMonth_year_complaint[Customer_summary_Output_By_FailMonth_year_complaint.loc[:,"Mil<1000 Qty"] > KM_COMPLAINT]
# Customer_summary_Output_By_FailMonth_year_complaint = Customer_summary_Output_By_FailMonth_year_complaint.reset_index().sort_values("Customer_Name")
# Customer_summary_Output_By_FailMonth_year_complaint = Customer_summary_Output_By_FailMonth_year_complaint[Customer_summary_Output_By_FailMonth_year_complaint.month == Customer_summary_Output_By_FailMonth_year_complaint.current_month]
# Customer_summary_Output_By_FailMonth_year_complaint.to_excel("Customer_summary_Output_By_FailMonth_year_complaint.xlsx")
# for customer in Customer_summary_Output_By_FailMonth_year_complaint.Customer_Name.unique():
#     customer_df = Customer_summary_Output_By_FailMonth_year_complaint[Customer_summary_Output_By_FailMonth_year_complaint.Customer_Name == customer]
#     report_text = "\n"
#     report_list= []
#     for json_data in json.loads(customer_df.to_json(orient='records')):
#         Customer_Name = json_data["Customer_Name"]
#         Product_Name = json_data["Product_Name"]
#         current_year = json_data["year"]
#         current_month = json_data["month"]
#         failure_quantity =json_data["Mil<1000 Qty"]
#         report_list.append(f"{Customer_Name}客户 {Product_Name}产品{current_year}年{current_month}月 1000KM失效={failure_quantity}")
#         report_tuple = tuple(report_list)
#     report_text = report_text.join(report_tuple)
#     send_report(Subject="1000KM Complaint Warning",content_1=f"您好:\n {report_text}, 已超过公司质量目标，请调查原因及制定改善措施，谢谢。")
#     time.sleep(10)

Customer_summary_Output_By_FailMonth_year_complaint

KeyError: 'Statistic month'

In [3]:
Customer_summary_Output_By_FailMonth_year_complaint[Customer_summary_Output_By_FailMonth_year_complaint.index.get_level_values("Customer_Name") == "FOTON"]

Mil<1000 Qty  current_year  \
Customer_Name Product_Name year month                               
FOTON         CN6 DPF      2025 1                 0          2025   
                                3                 0          2025   
                                4                 7          2025   
                                5                 0          2025   
              CN6 SCR      2025 2                 7          2025   
              Clamp        2025 5                 0          2025   
              DM2.2        2025 3                 0          2025   
                                4                 0          2025   
                                5                 1          2025   
              DM3.4        2025 5                 0          2025   
              DPS          2025 2                 0          2025   
                                3                 0          2025   
                                5                 1          2025   
              ET sensor    2025 1                 0          2025   
                                2                 0          2025   
                                3                 0          2025   
                                4                 0          2025   
                                5                 0          2025   
              GCU          2025 2                 0          2025   
                                3                 0          2025   
              GLP          2025 2                 0          2025   
                                3                 0          2025   
                                4                 0          2025   
              NX2          2025 1                 0          2025   
                                2                 0          2025   
                                3                 3          2025   
                                4                 1          2025   
                                5                 0          2025   
              PM2.1        2025 4                 1          2025   
                                5                 0          2025   
              SM5.3        2025 1                 0          2025   
                                2                 0          2025   
                                4                 0          2025   
              SM6-5        2025 3                 0          2025   
                                4                 1          2025   
                                5                 1          2025   

                                       current_month  
Customer_Name Product_Name year month                 
FOTON         CN6 DPF      2025 1                  6  
                                3                  6  
                                4                  6  
                                5                  6  
              CN6 SCR      2025 2                  6  
              Clamp        2025 5                  6  
              DM2.2        2025 3                  6  
                                4                  6  
                                5                  6  
              DM3.4        2025 5                  6  
              DPS          2025 2                  6  
                                3                  6  
                                5                  6  
              ET sensor    2025 1                  6  
                                2                  6  
                                3                  6  
                                4                  6  
                                5                  6  
              GCU          2025 2                  6  
                                3                  6  
              GLP          2025 2                  6  
                                3                  6  
                                4                  6  
              NX2          2025 1        

In [4]:
Customer_summary_Output_By_FailMonth = pd.read_excel("../Final_Output_folder/Customer_summary_Output_By_FailMonth.xlsx")
Customer_summary_Output_By_FailMonth = Customer_summary_Output_By_FailMonth[Customer_summary_Output_By_FailMonth.Fail_Month.notna()]
Customer_summary_Output_By_FailMonth = Customer_summary_Output_By_FailMonth[Customer_summary_Output_By_FailMonth.TNS_QTY.notna()]
Customer_summary_Output_By_FailMonth = Customer_summary_Output_By_FailMonth[Customer_summary_Output_By_FailMonth.Totol_Cost_Sum.notna()]
Customer_summary_Output_By_FailMonth.loc[:,"year"] = Customer_summary_Output_By_FailMonth.loc[:,"Fail_Month"].apply(lambda x:int(str(x)[:4]))
Customer_summary_Output_By_FailMonth.loc[:,"month"] = Customer_summary_Output_By_FailMonth.loc[:,"Fail_Month"].apply(lambda x:int(str(x)[5:7]))
Customer_summary_Output_By_FailMonth_exchange = Customer_summary_Output_By_FailMonth[["Customer_Name","Product_Name","Fail QYT","year","month"]].groupby(by=["Customer_Name","Product_Name","year","month"]).sum()
Customer_summary_Output_By_FailMonth_exchange.loc[:,"current_year"] = date.today().year
Customer_summary_Output_By_FailMonth_exchange.loc[:,"current_month"] = date.today().month
Customer_summary_Output_By_FailMonth_exchange = Customer_summary_Output_By_FailMonth_exchange[(Customer_summary_Output_By_FailMonth_exchange.index.get_level_values("year") == Customer_summary_Output_By_FailMonth_exchange.current_year) | (Customer_summary_Output_By_FailMonth_exchange.index.get_level_values("year") == Customer_summary_Output_By_FailMonth_exchange.current_year - 1)]

Customer_summary_Output_By_FailMonth_exchange = Customer_summary_Output_By_FailMonth_exchange.reset_index().sort_values("Customer_Name")

Customer_Name = list(Customer_summary_Output_By_FailMonth_exchange.Customer_Name.unique())
Product_Name = list(Customer_summary_Output_By_FailMonth_exchange.Product_Name.unique())
year = list(Customer_summary_Output_By_FailMonth_exchange.year.unique())
month = [x for x in range(1,13)]
master_df = pd.DataFrame(list(product(Customer_Name, Product_Name, year, month)), columns=["Customer_Name", "Product_Name", "year", "month"])
Customer_summary_Output_By_FailMonth_exchange = Customer_summary_Output_By_FailMonth_exchange.merge(master_df,on=["Customer_Name", "Product_Name", "year", "month"],how="right")
Customer_summary_Output_By_FailMonth_exchange = Customer_summary_Output_By_FailMonth_exchange.sort_values(["Customer_Name","Product_Name","year","month"])
Customer_summary_Output_By_FailMonth_exchange.loc[:,"Fail QYT"] = Customer_summary_Output_By_FailMonth_exchange.loc[:,"Fail QYT"].fillna(0)
Customer_summary_Output_By_FailMonth_exchange = Customer_summary_Output_By_FailMonth_exchange.ffill()
Customer_summary_Output_By_FailMonth_exchange = Customer_summary_Output_By_FailMonth_exchange.bfill()
Customer_summary_Output_By_FailMonth_exchange.loc[:,"incremental"] = Customer_summary_Output_By_FailMonth_exchange.groupby(["Customer_Name","Product_Name"])["Fail QYT"].diff()
# Customer_summary_Output_By_FailMonth_exchange[(Customer_summary_Output_By_FailMonth_exchange.loc[:,"Customer_Name"].isin(["ACPL"])) & (Customer_summary_Output_By_FailMonth_exchange.loc[:,"Product_Name"].isin(["DG"]))]
def filter_month(current_month : int):
    if date.today().day > 20:
        return current_month
    else:
        return [current_month - 2, current_month - 1]

Customer_summary_Output_By_FailMonth_exchange.loc[:,"current_month"] = Customer_summary_Output_By_FailMonth_exchange.loc[:,"current_month"].apply(lambda x:filter_month(x))

Customer_summary_Output_By_FailMonth_exchange_2month = Customer_summary_Output_By_FailMonth_exchange[Customer_summary_Output_By_FailMonth_exchange.month.isin(Customer_summary_Output_By_FailMonth_exchange.loc[:,"current_month"].drop_duplicates().values[0])]
Customer_summary_Output_By_FailMonth_exchange_2month = Customer_summary_Output_By_FailMonth_exchange_2month.groupby(["Customer_Name","year","Product_Name"]).min()[["incremental"]]
Customer_summary_Output_By_FailMonth_exchange_2month = Customer_summary_Output_By_FailMonth_exchange_2month[Customer_summary_Output_By_FailMonth_exchange_2month.index.get_level_values("year") == date.today().year]
Customer_summary_Output_By_FailMonth_exchange_2month = Customer_summary_Output_By_FailMonth_exchange_2month[Customer_summary_Output_By_FailMonth_exchange_2month.incremental > 0]
Customer_summary_Output_By_FailMonth_exchange_2month = Customer_summary_Output_By_FailMonth_exchange_2month.reset_index().sort_values("Customer_Name")


C:\Users\RBAdmin_App1\AppData\Local\Temp\4\ipykernel_22824\2049996290.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[list([5.0, 6.0]) list([5.0, 6.0]) list([5.0, 6.0]) ... list([5.0, 6.0])
 list([5.0, 6.0]) list([5.0, 6.0])]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  Customer_summary_Output_By_FailMonth_exchange.loc[:,"current_month"] = Customer_summary_Output_By_FailMonth_exchange.loc[:,"current_month"].apply(lambda x:filter_month(x))


In [ ]:
Customer_summary_Output_By_FailMonth_exchange_2month = Customer_summary_Output_By_FailMonth_exchange_2month.groupby(["Customer_Name","year","Product_Name"]).min()[["incremental"]]
Customer_summary_Output_By_FailMonth_exchange_2month = Customer_summary_Output_By_FailMonth_exchange_2month[Customer_summary_Output_By_FailMonth_exchange_2month.index.get_level_values("year") == date.today().year]
Customer_summary_Output_By_FailMonth_exchange_2month = Customer_summary_Output_By_FailMonth_exchange_2month[Customer_summary_Output_By_FailMonth_exchange_2month.incremental > -1]
Customer_summary_Output_By_FailMonth_exchange_2month = Customer_summary_Output_By_FailMonth_exchange_2month.reset_index().sort_values("Customer_Name")

,Customer_Name,year,Product_Name,incremental
0,ACPL,2025,BPS,0.0
22,ACPL,2025,MU,0.0
23,ACPL,2025,NGD-HFI,0.0
24,ACPL,2025,NX2,0.0
25,ACPL,2025,NX3,0.0
...,...,...,...,...
868,Yunnei,2025,ET sensor,0.0
869,Yunnei,2025,GCU,0.0
870,Yunnei,2025,GLP,0.0
872,Yunnei,2025,HFM,0.0


In [ ]:
# Customer_summary_Output_By_FailMonth_exchange
# def filter_last_12month(df, x):
#     temp_df = df[df]

Customer_summary_Output_By_FailMonth_exchange_12month = Customer_summary_Output_By_FailMonth_exchange[Customer_summary_Output_By_FailMonth_exchange.month == max((Customer_summary_Output_By_FailMonth_exchange.loc[:,"current_month"].drop_duplicates().values[0]))]

,Customer_Name,Product_Name,year,month,Fail QYT,current_year,current_month,incremental
192,ACPL,BPS,2024,1,0.0,2025.0,"[5.0, 6.0]",NaN
193,ACPL,BPS,2024,2,0.0,2025.0,"[5.0, 6.0]",0.0
194,ACPL,BPS,2024,3,0.0,2025.0,"[5.0, 6.0]",0.0
195,ACPL,BPS,2024,4,0.0,2025.0,"[5.0, 6.0]",0.0
196,ACPL,BPS,2024,5,0.0,2025.0,"[5.0, 6.0]",0.0
...,...,...,...,...,...,...,...,...
23011,Yunnei,Xin1,2025,8,0.0,2025.0,"[5.0, 6.0]",0.0
23012,Yunnei,Xin1,2025,9,0.0,2025.0,"[5.0, 6.0]",0.0
23013,Yunnei,Xin1,2025,10,0.0,2025.0,"[5.0, 6.0]",0.0
23014,Yunnei,Xin1,2025,11,0.0,2025.0,"[5.0, 6.0]",0.0


In [12]:
Customer_summary_Output_By_FailMonth_exchange.month.isin(Customer_summary_Output_By_FailMonth_exchange.loc[:,"current_month"].drop_duplicates().values[0]).value_counts()

month
False    19320
True      3864
Name: count, dtype: int64

In [49]:
Customer_summary_Output_By_FailMonth_exchange[(Customer_summary_Output_By_FailMonth_exchange.Customer_Name == "JMC") & (Customer_summary_Output_By_FailMonth_exchange.year == 2025)][-20:]

,Customer_Name,Product_Name,year,month,Fail QYT,current_year,current_month,incremental
15520,JMC,VCU,2025,5,0.0,2025.0,6.0,0.0
15521,JMC,VCU,2025,6,0.0,2025.0,6.0,0.0
15522,JMC,VCU,2025,7,0.0,2025.0,6.0,0.0
15523,JMC,VCU,2025,8,0.0,2025.0,6.0,0.0
15524,JMC,VCU,2025,9,0.0,2025.0,6.0,0.0
15525,JMC,VCU,2025,10,0.0,2025.0,6.0,0.0
15526,JMC,VCU,2025,11,0.0,2025.0,6.0,0.0
15527,JMC,VCU,2025,12,0.0,2025.0,6.0,0.0
15948,JMC,Xin1,2025,1,0.0,2025.0,6.0,-2.0
15949,JMC,Xin1,2025,2,2.0,2025.0,6.0,2.0


In [30]:
report_text

'NGD-HFI客户,2025年2月YTD EDC=6'

In [17]:
json.loads(Customer_summary_Output_By_FailMonth_year_complaint.to_json(orient='records'))

[{'Customer_Name': 'BFCEC',
  'Product_Name': 'NGD-HFI',
  'year': 2025,
  'month': 2,
  'Mil<1000 Qty': 6},
 {'Customer_Name': 'FOTON',
  'Product_Name': 'PM2.5',
  'year': 2024,
  'month': 6,
  'Mil<1000 Qty': 3},
 {'Customer_Name': 'FOTON',
  'Product_Name': 'PM2.1',
  'year': 2022,
  'month': 4,
  'Mil<1000 Qty': 4},
 {'Customer_Name': 'FOTON',
  'Product_Name': 'PM2.1',
  'year': 2021,
  'month': 11,
  'Mil<1000 Qty': 4},
 {'Customer_Name': 'FOTON',
  'Product_Name': 'PM2.1',
  'year': 2021,
  'month': 9,
  'Mil<1000 Qty': 3},
 {'Customer_Name': 'FOTON',
  'Product_Name': 'PM2.1',
  'year': 2020,
  'month': 9,
  'Mil<1000 Qty': 3},
 {'Customer_Name': 'FOTON',
  'Product_Name': 'NX2',
  'year': 2023,
  'month': 9,
  'Mil<1000 Qty': 3},
 {'Customer_Name': 'FOTON',
  'Product_Name': 'NX2',
  'year': 2023,
  'month': 1,
  'Mil<1000 Qty': 3},
 {'Customer_Name': 'FOTON',
  'Product_Name': 'PM2.1',
  'year': 2022,
  'month': 5,
  'Mil<1000 Qty': 5},
 {'Customer_Name': 'FOTON',
  'Product

In [18]:
Customer_summary_Output_By_FailMonth

,Customer_Name,Material,Product_Type,Product_Name,Fail_Month,Fail QYT,Totol_Cost_Sum,TNS_QTY,TNS,Identify,TNS_QTY_Year,TNS_Year,Mil<1000 Qty,year,month
4,ACPL,0281.006.277,SU,HFM,2025-02,9,0.00,1800.0,274050.00,ACPL&&0281.006.277&&2025-02,7650.0,1112512.50,0,2025,2
8,ACPL,0281.002.315,SU,DG,2025-02,3,0.00,2400.0,57960.00,ACPL&&0281.002.315&&2025-02,14400.0,331200.00,0,2025,2
9,ACPL,0281.002.315,SU,DG,2025-01,5,0.00,2400.0,57960.00,ACPL&&0281.002.315&&2025-01,14400.0,331200.00,0,2025,1
11,ACPL,0281.002.315,SU,DG,2024-12,2,0.00,2400.0,57960.00,ACPL&&0281.002.315&&2024-12,44400.0,1072260.00,0,2024,12
17,ACPL,0281.006.277,SU,HFM,2025-03,4,0.00,2250.0,290362.50,ACPL&&0281.006.277&&2025-03,7650.0,1112512.50,0,2025,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12174,Weichai_WestPort,0261.230.499,SU,BPS,2025-05,1,526.92,34250.0,2055000.00,Weichai_WestPort&&0261.230.499&&2025-05,84000.0,3235800.60,0,2025,5
12175,Weichai_WestPort,F01Z.F00.10B,AF,TVA-CQ,2025-05,2,657.60,7056.0,5372944.08,Weichai_WestPort&&F01Z.F00.10B&&2025-05,12432.0,9512464.08,0,2025,5
12181,Weichai_WestPort,0258.007.02Y,SU,LSU,2025-03,1,481.26,4000.0,1129200.00,Weichai_WestPort&&0258.007.02Y&&2025-03,17800.0,5024940.00,0,2025,3
12183,Weichai_WestPort,F01Z.F10.00H,AF,IGC,2025-03,1,40.91,6048.0,5322240.00,Weichai_WestPort&&F01Z.F10.00H&&2025-03,14784.0,13009920.00,0,2025,3


In [7]:
Customer_summary_Output_By_FailMonth_year_complaint.reset_index().sort_values("Customer_Name")

,Customer_Name,Product_Name,year,Mil<1000 Qty
0,ACPL,NX2,2024,3
1,BFCEC,NGD-HFI,2025,6
23,FOTON,SM6-5,2023,7
22,FOTON,SM6-5,2022,7
21,FOTON,SM5.3,2024,5
...,...,...,...,...
84,Qingling,CN5 Urea tank,2020,4
95,Qingling,HFM,2021,3
109,SFH,DG,2020,17
108,SFH,BPS,2020,34
